In [160]:
import numpy as np
import pandas as pd
import json

import ipywidgets as widgets
from IPython.display import display

import matplotlib.pyplot as plt
import altair as alt

In [2]:
def read_file(path):
    return pd.io.json.read_json(get_path(path))


get_path = lambda path: f'./json/{path}.json'
files = ["book", "patron", "rent"]
dfs = dict(zip(files, map(read_file, files)))

In [121]:
library = {}
raws = dfs['rent']['library'].unique()
i = 0

output = widgets.Output()
raw_html = widgets.HTML(value=raws[i])
dropdown = widgets.Dropdown(
    options=faculties.keys(),
    description='Faculty:',
    disabled=False,
)
button = widgets.Button(description='confirm')
text = widgets.Text()

def confirm():
    global library
    global raws
    global i
    raw_faculty = raws[i]
    i += 1
    print(text.value)
    if text.value:
        if text.value in library:
            library[text.value].append(raw_faculty)
        else:
            library[text.value] = [raw_faculty]
    else:
        if dropdown.value in library:
            library[dropdown.value].append(raw_faculty)
        else:
            library[dropdown.value] = [raw_faculty]
            
    text.value = ''
    raw_html.value = raws[i]
    dropdown.options = faculties.keys()
    output.clear_output()
    with output:
        display(raw_html, dropdown, text, button)


button.on_click(lambda x: confirm())
output.append_display_data(raw_html)
output.append_display_data(dropdown)
output.append_display_data(text)
output.append_display_data(button)

HTML(value='Veterinary Science Library')

Dropdown(description='Faculty:', options=('vet', 'staff', 'science', 'architecture', 'art', 'law', 'medicine',…

Text(value='')

Button(description='confirm', style=ButtonStyle())

In [138]:
faculties_map = {}
for k,v in faculties.items():
    for x in v:
        faculties_map[x] = k
        
library_map = {}
for k,v in library.items():
    for x in v:
        library_map[x] = k

In [186]:
dfs['rent']['faculty_map'] = dfs['rent']['faculty'].map(faculties_map)
dfs['rent']['library_map'] = dfs['rent']['library'].map(library_map)
dfs['rent'].head()

,id,faculty,library,title,day,month,year,period,faculty_map,library_map
0,0,อ.คณะสัตวแพทย์ฯ,Veterinary Science Library,Miller's guide to the dissection of the dog,Fri,4,2011,morning,vet,vet
1,1,อ.คณะสัตวแพทย์ฯ,Veterinary Science Library,Small Animal Dentistry,Thu,7,2010,afternoon,vet,vet
2,2,อ.คณะสัตวแพทย์ฯ,Veterinary Science Library,Clinical sciences,Mon,7,2010,afternoon,vet,vet
3,3,อ.คณะสัตวแพทย์ฯ,Veterinary Science Library,The 5 minute Veterinary consult clinical compa...,Fri,6,2010,morning,vet,vet
4,4,อ.คณะสัตวแพทย์ฯ,Veterinary Science Library,Managing a veterinary practice,Wed,3,2011,morning,vet,vet


In [185]:
data = (dfs['rent']
     .groupby(['faculty_map', 'library_map'])
     .size()
     .reset_index()
     .sort_values(['faculty_map', 0], ascending=[1, 0])
     .rename({ 0: 'count' }, axis=1)
)
data.head()
# .sort_values(ascending=False).sort_index(level=0)

,faculty_map,library_map,count
2,agricultural,staff,199
0,agricultural,art,2
1,agricultural,education,2
19,architecture,staff,4662
3,architecture,architecture,4152


In [184]:
alt.Chart.from_dict({
    "data": { "values": data.to_json(orient='records') },
    "mark": { "type": "bar", "tooltip": True },
    "encoding": {
        "x": {
            "field": "faculty_map",
            "type": "nominal",
            "sort": "-y"
        },
        "y": {
            "field": "count",
            "type": "quantitative"
        },
        "color": {
            "field": "library_map",
            "type": "nominal",
        }
    }
})

alt.Chart(...)

In [187]:
faculties

{'vet': ['อ.คณะสัตวแพทย์ฯ',
  'นิสิตสัตวแพทย์',
  'น.บัณฑิตสัตวแพทย',
  'อ.ตปท.สัตวแพทย์',
  'ป.เอก สัตวแพทย์'],
 'staff': ['ข้าราชการ',
  'อาจารย์',
  'ลูกจ้าง',
  'บุคลากรห้องสมุด',
  'บ.ห้องสมุดต่างปร',
  'อ.สถาบันภาษา',
  'อาจารย์พิเศษ',
  'OAR Member A (Alumni)',
  'นักวิจัย',
  'นักวิจัยต่างประเ',
  'นิสิตปริญญาเอก',
  'OAR Member B (Alumni)',
  'สมาชิกข้าราชการบำนาญของสนง. ประเภท',
  'นิสิต',
  'นิสิตบัณฑิต',
  'ขรก.ต่างประเทศ',
  'ลูกจ้างต่างประ',
  'อาจารย์พิเศษตปท.',
  'ยืมระหว่างห้องสม',
  'Member A (Alumni)',
  'OAR Member B (Student)',
  'อาจารย์ตปท.',
  'หน่วยราชการอื่น',
  'fss E Interlibrary Loan',
  'fss E Corporate Member',
  'fss E Alumni Member',
  'นิสิตบัณฑิตตปท.',
  'OAR Member A (People)',
  'Test Patron',
  'หนังสือประกอบนิทรรศการ',
  'OAR Member A (E Alumni)',
  'OAR Member A (Student)',
  'ยืมระหว่างห้องสมุดต่างประเทศ',
  'หน่วยงานเอกชน'],
 'science': ['อ.คณะวิทยาศาสตร์',
  'ป.เอกวิทยาศาสตร์',
  'นิสิตวิทยาศาสตร์',
  'น.บัณฑิตวิทยาศาส'],
 'architecture': ['นิ